In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split



ModuleNotFoundError: No module named 'sklearn'

In [48]:
dat = pd.read_csv('data/Absenteeism_at_work.csv', sep=';')

In [49]:
dat.columns = dat.columns.str.lower()
dat.columns

Index(['id', 'reason for absence', 'month of absence', 'day of the week',
       'seasons', 'transportation expense', 'distance from residence to work',
       'service time', 'age', 'work load average/day ', 'hit target',
       'disciplinary failure', 'education', 'son', 'social drinker',
       'social smoker', 'pet', 'weight', 'height', 'body mass index',
       'absenteeism time in hours'],
      dtype='object')

In [50]:
dat.rename(columns={
    'reason for absence': 'reason',
    'month of absence': 'month',
    'day of the week': 'day',
    'transportation expense' : 'transport_expense',
    'distance from residence to work': 'distance',
    'service time': 'service_time',
    'work load average/day ': 'daily_avg_work_load',
    'hit target': 'hit_target',
    'disciplinary failure': 'disciplinary_failure',
    'son': 'num_children',
    'social drinker': 'social_drinker',
    'social smoker': 'social_smoker',
    'body mass index': 'bmi',
    'absenteeism time in hours': 'absent_hours'
}, inplace=True)

In [51]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    740 non-null    int64  
 1   reason                740 non-null    int64  
 2   month                 740 non-null    int64  
 3   day                   740 non-null    int64  
 4   seasons               740 non-null    int64  
 5   transport_expense     740 non-null    int64  
 6   distance              740 non-null    int64  
 7   service_time          740 non-null    int64  
 8   age                   740 non-null    int64  
 9   daily_avg_work_load   740 non-null    float64
 10  hit_target            740 non-null    int64  
 11  disciplinary_failure  740 non-null    int64  
 12  education             740 non-null    int64  
 13  num_children          740 non-null    int64  
 14  social_drinker        740 non-null    int64  
 15  social_smoker         7

In [52]:
dat = dat.drop(columns=["id"])

In [53]:
dat.head()

,reason,month,day,seasons,transport_expense,distance,service_time,age,daily_avg_work_load,hit_target,disciplinary_failure,education,num_children,social_drinker,social_smoker,pet,weight,height,bmi,absent_hours
0,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


## Random Forest Feature Selection

In [ ]:
# Assuming 'absent_hours' is your target variable
X = df.drop(['absent_hours', 'id'], axis=1)  # Remove target and ID column
y = df['absent_hours']

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)